# MiniProject - Week 2

## Main Goals
- build the database of restaurants, bars and various points of interest (POIs)
- determine which API has better coverage (in Halifax)
- find the best path to vist the top 10 Points of interest

## Project Components and Tasks:
1. Setup and Planning
2. Making API Calls
3. Creating pandas DataFrames
4. Creating SQL databases
5. Determine top 10 points of interest
6. Comparing the API results
7. Completing the Travelling Salesman Problem

---

**Task One** <br>
Setup and Planning <br>
- Pick an area of interest 
- Import relevant packages
- Store API keys

In [1]:
#import packages
import os 
import pandas as pd
import foursquare
import requests
import json
from IPython.display import JSON
import sqlite3 as sqlite

In [2]:
#Area of interest is Halifax, NS
lat = 44.650029
long = -63.571080
ll = str(lat)+","+str(long)

In [6]:
#Store API Keys
#start with foursquare client ID and client Secret
foursquare_id =  #use os.environ before posting
foursquare_secret = 
#set foursquare client to allow you to use the foursquare doumentation
client = foursquare.Foursquare(client_id=foursquare_id, client_secret=foursquare_secret,version='20200731')

#setup yelp with API key
yelp_api_key = ""
headers = {'Authorization': 'Bearer %s' % yelp_api_key}

---

**Task Two** <br>
Making API Calls <br>
- Review API documentation
- Set task specific parameters
- Store results for further analysis 

*Documentation Notes* <br>
We will use the same categories for both to allow us to compare results <br>
Categories include: coffee shops, breweries, resturants, comedy clubs

|Foursquare|Yelp|
| :---: | :---: | 
|using venue/search|businesses/search
|Key parameters include : radius, categoryid|Key parameters include : radius, category|


***Foursquare***

In [7]:
#store categoryId to use in calls
coffee = "4bf58dd8d48988d1e0931735"
brewery = "50327c8591d4c4b30a586d5d"
resturants = "4d4b7105d754a06374d81259"
comedy = "4bf58dd8d48988d18e941735"

#store a maximum radius 
radius = 10000

In [8]:
foursquare_coffee = client.venues.search(params={'radius': radius, 'categoryId':coffee, 'll': ll})
foursquare_brewery = client.venues.search(params={'radius': radius, 'categoryId':brewery, 'll': ll})
foursquare_resturants = client.venues.search(params={'radius': radius, 'categoryId':resturants, 'll': ll})
foursquare_comedy = client.venues.search(params={'radius': radius, 'categoryId':comedy, 'll': ll})

***Yelp***

In [39]:
#store categoryId to use in calls
coffee = "coffee"
brewery = "breweries"
resturants = "resturants"
comedy = "comedyclubs"

#store a maximum radius 
radius = 10000

#store standard url
url = "https://api.yelp.com/v3/businesses/search"

In [42]:
#perform calls
params_coffee = {'latitude':lat,'longitude':long,'radius':radius,'categories':coffee}
params_brew = {'latitude':lat,'longitude':long,'radius':radius,'categories':brewery}
params_resturants = {'latitude':lat,'longitude':long,'radius':radius,'categories':resturants}
params_comedy = {'latitude':lat,'longitude':long,'radius':radius,'categories':comedy}

yelp_coffee = requests.get(url, params=params_coffee, headers=headers).json()
yelp_brewery = requests.get(url, params=params_brew, headers=headers).json()
yelp_resturants = requests.get(url, params=params_resturants, headers=headers).json()
yelp_comedy = requests.get(url, params=params_comedy, headers=headers).json()

----

**Task Three** <br>
Creating pandas DataFrames
- use pandas methods such as json_normalize()
- combine the various API results into one DataFrame

***Foursquare***

In [31]:
def get_venue_info(response):
    """Extract the venues information portion of the file"""
    response_venues = response["venues"]
    return response_venues
    
def get_dataframe(response_venues):
    """returns the pandas dataframe"""
    df = pd.json_normalize(response_venues)
    df_cat = pd.json_normalize(response_venues, record_path ="categories")
    df_merged = pd.merge(df, df_cat, how='left', left_index = True, right_index = True, suffixes=("_venue","_category"))
    df_merged = df_merged.drop("categories",axis = 1)
    df_merged = df_merged.drop("location.labeledLatLngs",axis=1) 
    df_merged = df_merged.drop("location.formattedAddress", axis=1)
    return df_merged

In [32]:
foursquare_coffee_df = get_dataframe(get_venue_info(foursquare_coffee))
foursquare_brewery_df = get_dataframe(get_venue_info(foursquare_brewery))
foursquare_resturants_df = get_dataframe(get_venue_info(foursquare_resturants))
foursquare_comedy_df = get_dataframe(get_venue_info(foursquare_comedy))

In [34]:
foursquare_df = foursquare_coffee_df.copy()
foursquare_df = foursquare_df.append(foursquare_brewery_df, ignore_index=True)
foursquare_df = foursquare_df.append(foursquare_resturants_df, ignore_index=True)
foursquare_df = foursquare_df.append(foursquare_comedy_df, ignore_index=True)

***Yelp***

In [46]:
def get_business_info(response):
    """Extract the business information portion of the file"""
    response_business = response["businesses"]
    return response_business
    
def get_dataframe(response_business):
    """returns the pandas dataframe"""
    df = pd.json_normalize(response_business)
    df_cat = pd.json_normalize(response_business, record_path ="categories")
    df_merged = pd.merge(df, df_cat, how='left', left_index = True, right_index = True, suffixes=("_venue","_category"))
    df_merged = df_merged.drop("categories",axis = 1)
    df_merged = df_merged.drop("transactions",axis=1) 
    df_merged = df_merged.drop("location.display_address", axis=1)
    return df_merged

In [47]:
yelp_coffee_df = get_dataframe(get_business_info(yelp_coffee))
yelp_brewery_df = get_dataframe(get_business_info(yelp_brewery))
yelp_resturants_df = get_dataframe(get_business_info(yelp_resturants))
yelp_comedy_df = get_dataframe(get_business_info(yelp_comedy))

In [48]:
yelp_df = yelp_coffee_df.copy()
yelp_df = yelp_df.append(yelp_brewery_df, ignore_index=True)
yelp_df = yelp_df.append(yelp_resturants_df, ignore_index=True)
yelp_df = yelp_df.append(yelp_comedy_df, ignore_index=True)

----

**Task Four** <br>
Creating SQL databases
- using sqlite3
- using pandas method to_sql()

In [36]:
#function .create_connection() that accepts the path to the SQLite database.
def create_connection(path):
    connection = None
    try:
    #uses .connect() from the sqlite3 module and takes the SQLite database path as a parameter. 
    #If the database exists at the specified location, then a connection to the database is established. 
    #Otherwise, a new database is created at the specified location, and a connection is established.   
        connection = sqlite.connect(path)
        print("Connection to SQLite DB successful")
    #catches any exception that might be thrown if .connect() fails to establish a connection.
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

***Foursquare***

In [49]:
connection = create_connection("foursquare.db")

Connection to SQLite DB successful


In [50]:
df_foursquare = foursquare_df.copy()
df_foursquare.to_sql('poi', con = connection, if_exists = 'replace', )

***Yelp***

In [51]:
connection = create_connection("yelp.db")

Connection to SQLite DB successful


In [52]:
df_yelp = yelp_df.copy()
df_yelp.to_sql('poi', con = connection, if_exists = 'replace', )

---

***Task 5*** <br>
Determine top 10 points of interest
- Foursquare obtain ratings for a sample of venue ids
- Yelp observe the exisiting data and ratings
- Yelp perform new API call with param sortby=rating

In [53]:
yelp_topten = yelp_df.sort_values(by="rating").head(10)

In [54]:
#make a call to determine top 10 POIs
sortby = "rating"
limit ="10"
params_top = {'latitude':lat,'longitude':long,'radius':radius,'sort_by':sortby, 'limit':limit}
yelp_top = requests.get(url, params=params_top, headers=headers).json()

In [55]:
JSON(yelp_top)

<IPython.core.display.JSON object>

> In the yelp documentation it explains that the sortby is "not strictly enforced". This means that it is not just returning the highest rating because it is also weighting the number of reviews. This is why we have some higher rated items in our inital results 

In [56]:
def get_rating (venue_id):
    """return the rating for a given venue id if available"""
    venue_id = str(venue_id)
    foursquare_id = ""
    foursquare_secret = 
    client = foursquare.Foursquare(client_id=foursquare_id, client_secret=foursquare_secret,version='20200731')
    res = client.venues(venue_id)
    try:
        rating =res["venue"]["rating"]
        return rating
    except KeyError: 
        return None

In [57]:
sample_foursquare=foursquare_df.sample(15)

In [58]:
sample_foursquare['rating'] = sample_foursquare['id_venue'].apply(get_rating)

In [59]:
foursquare_topten = sample_foursquare.sort_values(by="rating",ascending=False).head(10)

> In the foursquare documentation it states that it will not return ratings for venues that don't have reviews -- this resulted in missing information 

----

**Task Six** <br>
Comparison

In [37]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [60]:
print ("the length of" + color.BOLD + ' foursquare results ' + color.END + "is: {}".format(len(foursquare_df)))
print ("the length of" + color.BOLD + ' yelp results ' + color.END + "is: {}".format(len(yelp_df)))
duplicateRowsDF_foursquare = foursquare_df[foursquare_df.duplicated(['id_venue'])]
duplicateRowsDF_yelp = yelp_df[yelp_df.duplicated(['id'])]

print ("the number of duplicate" + color.BOLD + ' foursquare results ' + color.END + "is: {}".format(len(duplicateRowsDF_foursquare)))
print ("the number of duplicate" + color.BOLD + ' yelp results ' + color.END + "is: {}".format(len(duplicateRowsDF_yelp)))

the length of foursquare results is: 95
the length of yelp results is: 61
the number of duplicate foursquare results is: 12
the number of duplicate yelp results is: 1


In [61]:
categoryCount_foursquare = foursquare_df.groupby("name_category")["name_venue"].count()
print(categoryCount_foursquare)
categoryCount_yelp = yelp_df.groupby("alias_category")["name"].count()
print(categoryCount_yelp)

name_category
Breakfast Spot           1
Brewery                 25
Burger Joint             1
Coffee Shop             41
Comedy Club             10
Fast Food Restaurant     4
Food Court               1
Gastropub                1
Grocery Store            4
Italian Restaurant       1
Pizza Place              2
Pub                      1
Restaurant               1
Sandwich Place           1
Supermarket              1
Name: name_venue, dtype: int64
alias_category
bakeries             5
bars                 3
bbq                  1
beertours            1
bookstores           1
breakfast_brunch     2
breweries           14
brewpubs             3
burgers              1
cocktailbars         1
coffee              11
comedyclubs          1
desserts             2
gluten_free          1
italian              1
lounges              1
musicvenues          1
newcanadian          3
pizza                1
pubs                 2
sandwiches           1
tea                  1
vegan                1
wine_b

In [130]:
foursquare_df[foursquare_df["name_category"] == "Coffee Shop"]["name_venue"]

0           Tim Hortons
1             Starbucks
2             Starbucks
3               Almonak
4           Tim Hortons
5           Tim Hortons
6           Tim Hortons
7             Starbucks
8             Starbucks
9           Tim Hortons
10          Tim Hortons
11          Tim Hortons
12            Starbucks
13          Tim Hortons
14            Starbucks
15          Tim Hortons
16          Tim Hortons
17            Nespresso
18          Tim Hortons
19           Second Cup
20           Second Cup
21     Uncommon Grounds
22            Starbucks
23          Tim Hortons
24            Starbucks
25           Second Cup
26    Java Blend Coffee
27         Cabin Coffee
28       The Wired Monk
55          Tim Hortons
63            Starbucks
65            Starbucks
67            Starbucks
71          Tim Hortons
77          Tim Hortons
79            Starbucks
80          Tim Hortons
81           Second Cup
82    Java Blend Coffee
83            Starbucks
84       The Wired Monk
Name: name_venue

In [126]:
yelp_df[yelp_df["alias_category"]=="coffee"]["name"]

0       Java Blend Coffee Roasters
1         The Board Room Game Cafe
2                  World Tea House
4                     Cabin Coffee
6     The Wired Monk Coffee Bistro
9        Steve O Reno's Cappuccino
11                Uncommon Grounds
15         Dilly Dally Coffee Cafe
16                 Narrow Espresso
19                         Chatime
56                    Cha Baa Thai
Name: name, dtype: object

|Element|Foursquare|Yelp|
| :---: | :---: | :---: | 
|Response Number|Returned **More** Responses|Returned **Less** Responses|
|Duplication in Responses|Returned **More** Duplicates|Returned **Less** Duplicates|
|Category| Provided **Less Specific** Categories| Provide More Categories and **More Specific** Categories|
|Quality of Results|More **Generic** Options|More **Local Specific** Options|
|Ratings| **Less** Ratings Provided| **More** Ratings Provided|

-----

**Task Seven** <br>
Completing the Travelling Salesman Problem

*Notes* : Code for the matrix and TSP was taken from the Google API instructions in order to obtain the response to the TSP. <br>as provided [here](https://developers.google.com/optimization/routing/tsp)

***Get Addresses of Top 10 places to set as Destinations***

In [63]:
def get_addresses(civic_address2,civic_address1, city, province):
    """function to extract address from our DataFrame"""
    if civic_address2 == "":
        ca = civic_address1.split(" ")
    elif civic_address2 != None:
        ca = civic_address2.split(" ")
    else:
        ca = civic_address1.split(" ")
    
    for i in range(len(ca)):
        if i == 0:
            street_address = ca[i]
        else:
            street_address = str(street_address) + "+" + str(ca[i])
    
    address = str(street_address)+"+"+str(city)+"+"+str(province)
    return address

In [64]:
#apply the function to our dataframe of POIs
addresses = yelp_topten.apply(lambda x: get_addresses(x['location.address2'], x['location.address1'],
                                                 x['location.city'],x['location.state']), axis=1)

In [65]:
#Convert the output to a list
address = addresses.tolist()

***Determine Distance Matrix***

In [66]:
#FROM API DOCUMENTATION
def build_distance_matrix(response):
    distance_matrix = []
    for row in response['rows']:
        row_list = [row['elements'][j]['distance']['value'] for j in range(len(row['elements']))]
        distance_matrix.append(row_list)
    return distance_matrix

In [67]:
#FROM API DOCUMENTATION
def send_request(origin_addresses, dest_addresses, API_key):
    """ Build and send request for the given origin and destination addresses."""
    def build_address_str(addresses):
        # Build a pipe-separated string of addresses
        address_str = ''
        for i in range(len(addresses) - 1):
            address_str += addresses[i] + '|'
        address_str += addresses[-1]
        return address_str

    request = 'https://maps.googleapis.com/maps/api/distancematrix/json?units=metric'
    origin_address_str = build_address_str(origin_addresses)
    dest_address_str = build_address_str(dest_addresses)
    request = request + '&origins=' + origin_address_str + '&destinations=' + \
                       dest_address_str + '&key=' + API_key
    response = requests.get(request).json()
    return response

In [68]:
#FROM API DOCUMENTATION 
def create_distance_matrix(data):
    addresses = data["addresses"]
    API_key = data["API_key"]
  
    # Distance Matrix API only accepts 100 elements per request, so get rows in multiple requests.
    max_elements = 100
    num_addresses = len(addresses) # 16 in this example.
    # Maximum number of rows that can be computed per request (6 in this example).
    max_rows = max_elements // num_addresses
    # num_addresses = q * max_rows + r (q = 2 and r = 4 in this example).
    q, r = divmod(num_addresses, max_rows)
    dest_addresses = addresses
    distance_matrix = []
    # Send q requests, returning max_rows rows per request.
    for i in range(q):
        origin_addresses = addresses[i * max_rows: (i + 1) * max_rows]
        response = send_request(origin_addresses, dest_addresses, API_key)
        distance_matrix += build_distance_matrix(response)
    # Get the remaining remaining r rows, if necessary.
    if r > 0:
        origin_addresses = addresses[q * max_rows: q * max_rows + r]
        response = send_request(origin_addresses, dest_addresses, API_key)
        distance_matrix += build_distance_matrix(response)
    return distance_matrix

In [69]:
data = {'addresses':address, 'API_key':""}
distance_matrix = create_distance_matrix(data)

In [88]:
print(distance_matrix)

[[0, 3796, 2338, 1896, 2336, 3309, 0, 2194, 1569, 3027], [3767, 0, 1483, 2027, 1385, 657, 3767, 1403, 2099, 1142], [2512, 1655, 0, 1129, 747, 1015, 2512, 1067, 1383, 1195], [2238, 2057, 1129, 0, 628, 1393, 2238, 486, 360, 1129], [2878, 1270, 918, 829, 0, 783, 2878, 341, 1083, 501], [3110, 1076, 931, 1370, 728, 0, 3110, 746, 1442, 485], [0, 3796, 2338, 1896, 2336, 3309, 0, 2194, 1569, 3027], [2336, 1693, 866, 514, 365, 1130, 2336, 0, 769, 866], [1650, 2206, 1379, 364, 877, 1643, 1650, 735, 0, 1378], [2798, 1213, 1067, 749, 599, 726, 2798, 261, 1003, 0]]


***Complete Travelling Salesman Problem***

In [70]:
from __future__ import print_function
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [105]:
def print_solution(manager, routing, solution):
    """Prints solution on console."""
    print('Objective: {}'.format(solution.ObjectiveValue()))
    index = routing.Start(0)
    plan_output = 'Route:\n'
    route_distance = 0
    while not routing.IsEnd(index):
        plan_output += ' {} ->'.format(manager.IndexToNode(index))
        previous_index = index
        index = solution.Value(routing.NextVar(index))
        route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
    plan_output += ' {}\n'.format(manager.IndexToNode(index))
    print(plan_output)
    plan_output += 'Objective: {}metres\n'.format(route_distance)

In [106]:
TSP_data = {}
TSP_data['distance_matrix'] = distance_matrix #as found above
TSP_data['num_vehicles'] = 1 #Traveling sales man problem
TSP_data['depot'] = 9 #start at most popular

In [112]:
manager = pywrapcp.RoutingIndexManager(len(TSP_data['distance_matrix']),TSP_data['num_vehicles'], TSP_data['depot'])
routing = pywrapcp.RoutingModel(manager)

In [113]:
def distance_callback(from_index, to_index):
    """Returns the distance between the two nodes."""
    # Convert from routing variable Index to distance matrix NodeIndex.
    from_node = manager.IndexToNode(from_index)
    to_node = manager.IndexToNode(to_index)
    return data['distance_matrix'][from_node][to_node]

transit_callback_index = routing.RegisterTransitCallback(distance_callback)

In [114]:
routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

In [115]:
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

In [116]:
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(manager, routing, solution)

Objective: 0
Route:
 9 -> 8 -> 7 -> 6 -> 5 -> 4 -> 3 -> 2 -> 1 -> 0 -> 9

